In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

filepath_in = 'data/processed_data.csv'

2024-03-12 15:52:48.535743: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
df = pd.read_csv(filepath_in)
X = df[[col for col in df if col.startswith('emb_')]]
y = pd.get_dummies(df['label'])

In [4]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model architecture
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))  # Input layer
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dense(y_train.shape[1], activation='softmax'))  # Output layer

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
746/746 [==============================] - 2s 2ms/step - loss: 1.5928 - accuracy: 0.3498 - val_loss: 1.5176 - val_accuracy: 0.3834
Epoch 2/10
746/746 [==============================] - 1s 2ms/step - loss: 1.4852 - accuracy: 0.3937 - val_loss: 1.4692 - val_accuracy: 0.3964
Epoch 3/10
746/746 [==============================] - 1s 2ms/step - loss: 1.4451 - accuracy: 0.4139 - val_loss: 1.4337 - val_accuracy: 0.4179
Epoch 4/10
746/746 [==============================] - 1s 2ms/step - loss: 1.4151 - accuracy: 0.4292 - val_loss: 1.4067 - val_accuracy: 0.4375
Epoch 5/10
746/746 [==============================] - 1s 2ms/step - loss: 1.3954 - accuracy: 0.4445 - val_loss: 1.3882 - val_accuracy: 0.4442
Epoch 6/10
746/746 [==============================] - 2s 3ms/step - loss: 1.3725 - accuracy: 0.4537 - val_loss: 1.3723 - val_accuracy: 0.4538
Epoch 7/10
746/746 [==============================] - 1s 2ms/step - loss: 1.3525 - accuracy: 0.4611 - val_loss: 1.3558 - val_accuracy: 0.4447
Epoch 